In [1]:
import requests
import time, random
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [46]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
# request_payload_file = '/data/Magicoder_payloads.txt'
request_payload_file = '/data/Promptshett_codegen_50.txt'
report_path = '/data/vLLM_50prompts.csv'
df = pd.DataFrame(columns=['Prompt_No', 'Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [11]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/magicoder/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
# request_payload_file = '/data/Magicoder_payloads.txt'
request_payload_file = '/data/Promptshett_codegen_50.txt'
report_path = '/data/vLLM_50prompts.csv'
df = pd.DataFrame(columns=['Prompt_No', 'Prompt', 'Response', 'Response_status', 'Time (seconds)'])
print(df)

Empty DataFrame
Columns: [Prompt_No, Prompt, Response, Response_status, Time (seconds)]
Index: []


In [12]:
prompt=["""You are an exceptional dbt analytics engineer writes dbt models, also creates very precise and accurate functional documentation to ensure any layman understands the code written by you.
follow instruction and response accuratly. finally only generate respose without instruction. 
 
@@Instruction
this is the sample dbt model
with customer_revenue as (
select
customer_id,
sum(amount) as total_revenue
from
{ ref('orders') }
group by 1
)
 
select
c.customer_id,
c.first_name,
c.last_name,
cr.total_revenue
from
{ source('master_data','customers') } c
left join customer_revenue cr on c.customer_id = cr.customer_id

@@response
Below is the sample dbt model documentation generated for above instruction dbt model:
 
* Overview: This dbt model calculates the total revenue per customer by leveraging the data from the orders model and customers source. 
* Dependencies:
* orders: dbt model that contains information about individual orders, including the customer_id and amount.
* customers: dbt source contains customer details, including customer_id, first_name, and last_name.
* Transformation:
* The CTE 'customer_revenue' aggregates the total revenue for each customer by summing the amount from the orders model. It utilizes the customer_id field for grouping.
* The final output includes customer details from the customers table (c) and their corresponding total revenue from the customer_revenue CTE (cr). The join is performed using the customer_id field.
* Example Usage:  The resulting model can be used for various analytical purposes, such as identifying high-value customers or generating reports on customer revenue.

@@ Instruction
create similar dbt model documentation for following dbt model code
 {% snapshot users_snapshot %}  
 
{{
  config(      
    target_schema='snapshots',      
    strategy='timestamp',      
    unique_key='id',      
    updated_at='updated_at'    
  )  
}}  
 
  select * 
  from {{ source('raw','users') }}
 
{% endsnapshot %}

@@ response""",
       "Generate a DBT model which gives me variants of the products which has highest sales",
       "Construct a DBT view model in the 'staging' schema, linking 'jaffle_shop' orders with payment information from 'stg_payments', summarizing successful payments per order, and merging with order details to handle potential missing payment scenarios.",
       "Define an incremental dbt model named user_activity_daily with a unique key on date_day for daily active users. Implement the 'delete+insert' strategy for incremental updates."
       ]

In [5]:
 payload = {"prompt": "Problem: Create an animated dropdown menu in CSS that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu).", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [13]:
payload = {"prompt": prompt, 
           "max_tokens": 2400,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [17]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = line.strip()
#             prompt = json.loads(line.strip())
            yield prompt
#             yield prompt['payload'

In [14]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, a,b= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt_No':b,'Prompt':a,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result,a,b
    return calculate_execution_time

In [15]:
def save_report(file_path, df):
    print("Saving report\n", df)
    df.to_csv(file_path, index=False)

In [16]:
@my_decorator
def inference(prompt_index, payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, json=payload_data)
#     print(f"Prompt {prompt_index}==> {payload_data['prompt']} \n {response.text}")
    print(f"Prompt {prompt_index}==> Response \n {response.text}")
    return response, payload_data['prompt'], prompt_index
    
        

In [43]:
def my_worker(thread, payload):
    prompts = read_payloads(request_payload_file)
    try:
        for line, prompt in enumerate(prompts,start=0):
            payload = {**payload, 'prompt': prompt}
            print(f"Prompting prompt {line+1}")
            a,b,c,_ = inference(line, payload)
            print(f"Finished Prompting prompt {line+1}")
    except Exception as e:
        print(a,b,c)
        print(f"Error in thread {thread} - prompt {line}: {e}")

In [44]:
#Model Inference(Request)
# if __name__ == "__main__":
#     global df
#     with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
#         futures = [executor.submit(my_worker, _, payload) for _ in range(30)]
#         concurrent.futures.wait(futures)
#     save_report(report_path, df)

In [45]:
# #Model Inference(Request) Same Payload
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    print(len(prompts))
    index=0
    global df
    for i in range(1000):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(inference, srno, {**payload, 'prompt': prompts[random.randint(0,49)]}) for srno in range(10)]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

50
loop 0+++++++++++++++++++++++++++++
Prompt 9==> Response 
 {"id":"cmpl-58d71c5ad4fe48b1877d35e1bb6f4a20","object":"text_completion","created":2200450,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nAssuming you have two tables:\n\n1. `employees` table with columns: `employee_id`, `name`, `country`, `client_id`\n2. `clients` table with columns: `client_id`, `client_name`, `country`\n\nThe SQL query would look something like this:\n\n```sql\nSELECT e.employee_id, e.name, e.country, c.client_name, c.country\nFROM employees e\nJOIN clients c ON e.client_id = c.client_id\nWHERE c.country = 'US'\n```\n\nThis SQL statement will join the `employees` and `clients` tables on the `client_id` column, and then filter the results to only include rows where the `country` of the `clients` table is 'US'. The resulting table will contain the `employee_id`, `name`, `country` of the employee, `client_name`

KeyboardInterrupt: 